In [1]:
from IPython.display import Markdown, display
from llama_index import SQLDatabase, ServiceContext
import sqlite3
from llama_index.llms import OpenAI
from llama_index.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index import VectorStoreIndex
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)
import os

db_dir = "files/db"
db_files = [os.path.join(db_dir, file) for file in os.listdir(db_dir) if file.endswith('.db')]

for db_file in db_files:
    engine = create_engine('sqlite:///' + db_file)
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    for table in tables:
        print(table[0])

    table_name = table[0]
    print(f"\nTable: {table_name} in {db_file}")
    metadata_obj = MetaData()
    print(metadata_obj)
    # Get table schema
    print("Schema:")
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    for column in schema:
        print(column)

    # Get the first five rows
    # print("\nFirst 5 rows:")
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
    rows = cursor.fetchall()
    for row in rows:
        print(row)

    llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
    service_context = ServiceContext.from_defaults(llm=llm)
    sql_database = SQLDatabase(engine)
    # set Logging to DEBUG for more detailed outputs
    table_node_mapping = SQLTableNodeMapping(sql_database)
    table_schema_objs = [
        (SQLTableSchema(table_name=str(table_name)))
    ]  # add a SQLTableSchema for each table

    obj_index = ObjectIndex.from_objects(
        table_schema_objs,
        table_node_mapping,
        VectorStoreIndex,
    )
    query_engine = SQLTableRetrieverQueryEngine(
        sql_database, obj_index.as_retriever(similarity_top_k=3)
    )
    response = query_engine.query("How many different proteins with lymphoid tissue?")
    display(Markdown(f"<b>{response}</b>"))
    # Close the cursor and connection
    cursor.close()
    conn.close()


ccle_cn_gistic

Table: ccle_cn_gistic in files/db/CCLEGisticCNDB.db
MetaData()
Schema:
(0, 'Hugo_Symbol', 'TEXT', 0, None, 0)
(1, 'CCLE_Name', 'TEXT', 0, None, 0)
(2, 'gistic_cn', 'REAL', 0, None, 0)
(3, 'DepMap_ID', 'TEXT', 0, None, 0)
('A1BG', 'DMS53_LUNG', 0.0, 'ACH-000698')
('A1BG', 'SW1116_LARGE_INTESTINE', 0.0, 'ACH-000489')
('A1BG', 'NCIH1694_LUNG', 0.0, 'ACH-000431')
('A1BG', 'P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 0.0, 'ACH-000707')
('A1BG', 'HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 0.0, 'ACH-000509')


<b>There are no different proteins associated with lymphoid tissue in the query results.</b>

ccle_mutation

Table: ccle_mutation in files/db/CCLEMutDB.db
MetaData()
Schema:
(0, 'Hugo_Symbol', 'TEXT', 0, None, 0)
(1, 'Entrez_Gene_Id', 'INTEGER', 0, None, 0)
(2, 'NCBI_Build', 'INTEGER', 0, None, 0)
(3, 'Chromosome', 'TEXT', 0, None, 0)
(4, 'Start_position', 'INTEGER', 0, None, 0)
(5, 'End_position', 'INTEGER', 0, None, 0)
(6, 'Strand', 'TEXT', 0, None, 0)
(7, 'Variant_Classification', 'TEXT', 0, None, 0)
(8, 'Variant_Type', 'TEXT', 0, None, 0)
(9, 'Reference_Allele', 'TEXT', 0, None, 0)
(10, 'Alternate_Allele', 'TEXT', 0, None, 0)
(11, 'dbSNP_RS', 'TEXT', 0, None, 0)
(12, 'dbSNP_Val_Status', 'TEXT', 0, None, 0)
(13, 'Genome_Change', 'TEXT', 0, None, 0)
(14, 'Annotation_Transcript', 'TEXT', 0, None, 0)
(15, 'DepMap_ID', 'TEXT', 0, None, 0)
(16, 'cDNA_Change', 'TEXT', 0, None, 0)
(17, 'Codon_Change', 'TEXT', 0, None, 0)
(18, 'Protein_Change', 'TEXT', 0, None, 0)
(19, 'isDeleterious', 'TEXT', 0, None, 0)
(20, 'isTCGAhotspot', 'TEXT', 0, None, 0)
(21, 'TCGAhsCnt', 'REAL', 0, None, 0

<b>There are no different proteins with lymphoid tissue.</b>

ccle_variants

Table: ccle_variants in files/db/CCLEVarDB.db
MetaData()
Schema:
(0, 'DepMap_ID', 'TEXT', 0, None, 0)
(1, 'Chrom', 'TEXT', 0, None, 0)
(2, 'Pos', 'INTEGER', 0, None, 0)
(3, 'Ref', 'TEXT', 0, None, 0)
(4, 'Alt', 'TEXT', 0, None, 0)
(5, 'AF', 'REAL', 0, None, 0)
(6, 'VariantType', 'TEXT', 0, None, 0)
(7, 'VariantInfo', 'TEXT', 0, None, 0)
(8, 'DNAChange', 'TEXT', 0, None, 0)
(9, 'ProteinChange', 'TEXT', 0, None, 0)
(10, 'gene_name', 'TEXT', 0, None, 0)
(11, 'Transcript', 'TEXT', 0, None, 0)
(12, 'CCLEDeleterious', 'TEXT', 0, None, 0)
(13, 'CosmicHotspot', 'TEXT', 0, None, 0)
(14, 'AssociatedWith', 'TEXT', 0, None, 0)
(15, 'LoF', 'TEXT', 0, None, 0)
(16, 'Driver', 'TEXT', 0, None, 0)
(17, 'LikelyDriver', 'TEXT', 0, None, 0)
(18, 'LikelyGoF', 'TEXT', 0, None, 0)
(19, 'LikelyLoF', 'TEXT', 0, None, 0)
(20, 'VariantAnnotation', 'TEXT', 0, None, 0)
(21, 'copy_number', 'TEXT', 0, None, 0)
('ACH-000839', 'chr1', 1242864, 'GC', 'CT', 0.31, 'DNP', 'MISSENSE', 'c.780_781GC>AG', 'p.L2

<b>There are no different proteins associated with lymphoid tissue.</b>

In [2]:
# Create combined schema of tables

import sqlite3
import json
from sqlalchemy import create_engine, MetaData, Table, Column, String
import re
def combine_schemas(db_files):
    combined_schema = {}

    for db_file in db_files:
        engine = create_engine('sqlite:///' + db_file)
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Get all tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        for table in tables:
            table_name = table[0]
            metadata_obj = MetaData()
            
            # Get table schema
            cursor.execute(f"PRAGMA table_info({table_name});")
            schema = cursor.fetchall()

            # Create a Table object to store schema info
            table_obj = Table(table_name, metadata_obj)

            for column in schema:
                col_name, col_type = column[1], column[2]
                # Add column to the table object
                table_obj.append_column(Column(col_name, String))

            # Serialize table schema
            schema_info = [{"column_name": col.name, "data_type": str(col.type)} for col in table_obj.columns]
            combined_schema[f"{table_name} in {db_file}"] = schema_info

        conn.close()

    return combined_schema

def save_schema_to_json(combined_schema, filename="combined_schema.json"):
    with open(filename, "w") as file:
        json.dump(combined_schema, file, indent=4)

# Paths to your database files
db_files = ["files/db/CCLEGisticCNDB.db" , 'files/db/CCLEMutDB.db', 'files/db/CCLEVarDB.db']
all_schemas = combine_schemas(db_files)
save_schema_to_json(all_schemas)

# Now the schema is saved in 'combined_schema.json'



Schema for ccle_cn_gistic in files/db/CCLEGisticCNDB.db:
ccle_cn_gistic.Hugo_Symbol
ccle_cn_gistic.CCLE_Name
ccle_cn_gistic.gistic_cn
ccle_cn_gistic.DepMap_ID

Schema for ccle_mutation in files/db/CCLEMutDB.db:
ccle_mutation.Hugo_Symbol
ccle_mutation.Entrez_Gene_Id
ccle_mutation.NCBI_Build
ccle_mutation.Chromosome
ccle_mutation.Start_position
ccle_mutation.End_position
ccle_mutation.Strand
ccle_mutation.Variant_Classification
ccle_mutation.Variant_Type
ccle_mutation.Reference_Allele
ccle_mutation.Alternate_Allele
ccle_mutation.dbSNP_RS
ccle_mutation.dbSNP_Val_Status
ccle_mutation.Genome_Change
ccle_mutation.Annotation_Transcript
ccle_mutation.DepMap_ID
ccle_mutation.cDNA_Change
ccle_mutation.Codon_Change
ccle_mutation.Protein_Change
ccle_mutation.isDeleterious
ccle_mutation.isTCGAhotspot
ccle_mutation.TCGAhsCnt
ccle_mutation.isCOSMIChotspot
ccle_mutation.COSMIChsCnt
ccle_mutation.ExAC_AF
ccle_mutation.Variant_annotation
ccle_mutation.CGA_WES_AC
ccle_mutation.HC_AC
ccle_mutation.RD_AC


In [ ]:
# A check
import re 
import logging

logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

def extract_sql(llm_response: str) -> str:
    # If the llm_response contains a markdown code block, with or without the sql tag, extract the sql from it
    sql = re.search(r"```sql\n(.*)```", llm_response, re.DOTALL)
    if sql:
        log(f"Output from LLM: {llm_response} \nExtracted SQL: {sql.group(1)}")
        return sql.group(1)

    sql = re.search(r"```(.*)```", llm_response, re.DOTALL)
    if sql:
        log(f"Output from LLM: {llm_response} \nExtracted SQL: {sql.group(1)}")
        return sql.group(1)

    return llm_response

def is_sql_valid(sql: str) -> bool:
    # This is a check to see the SQL is valid and should be run
    # This simple function just checks if the SQL contains a SELECT statement

    if "SELECT" in sql.upper():
        return True
    else:
        return False


In [ ]:
# LLM checks to write a SQL query

import replicate
import pandas as pd
import json
import os
from dotenv import load_dotenv
load_dotenv()
folder_path = 'files'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

with open('config.json', 'r') as config_file:
    config = json.load(config_file)

INSTRUCTION = config['instructions']
F_NAME = config["name"]

# # Load the file
df = pd.read_excel('files/questions_db.xlsx')
# Save the original DataFrame
df.to_excel('files/questions_original_db.xlsx', index=False)

# Trim whitespace and newline characters
df['Question'] = df['Question'].str.strip()  # Removes leading/trailing whitespace

# Check for duplicate questions
duplicates = df.duplicated(subset=['Question'], keep=False)
if duplicates.any():
    print("Duplicates found. Removing duplicates.")

    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=['Question'], keep='first')

    # Save the modified DataFrame, overwriting the original 'questions.xlsx'
    df.to_excel('files/questions_db.xlsx', index=False)
else:
    print("No duplicates found.")

# DataFrame to store the results
results_df = pd.DataFrame(columns=['Model', 'Question', 'Response'])

models = {
    "qwen-14b": "nomagick/qwen-14b-chat:f9e1ed25e2073f72ff9a3f46545d909b1078e674da543e791dec79218072ae70",
    "falcon-40b": "joehoover/falcon-40b-instruct:7d58d6bddc53c23fa451c403b2b5373b1e0fa094e4e0d1b98c3d02931aa07173",
    "yi-34b": "01-ai/yi-34b-chat:914692bbe8a8e2b91a4e44203e70d170c9c5ccc1359b283c84b0ec8d47819a46",
    "mistral-7b": "mistralai/mistral-7b-instruct-v0.2:f5701ad84de5715051cb99d550539719f8a7fbcf65e0e62a3d1eb3f94720764e",
    "llama2-70b": "meta/llama-2-70b-chat",
    "openhermes2": "antoinelyset/openhermes-2.5-mistral-7b:d7ccd25700fb11c1787c25b580ac8d715d2b677202fe54b77f9b4a1eb7d73e2b",
    "mixtral-instruct": "mistralai/mixtral-8x7b-instruct-v0.1:2b56576fcfbe32fa0526897d8385dd3fb3d36ba6fd0dbe033c72886b81ade93e",
    "deepseek_33bq": "kcaverly/deepseek-coder-33b-instruct-gguf:ea964345066a8868e43aca432f314822660b72e29cab6b4b904b779014fe58fd",
    }

prompt_for_qwen="""<|im_start|>system\n {INSTRUCTION}. Please write the appropriate SQL query using these three table schemas {all_schemas} to answer the following question. The SQL should be returned within ''' SQL query '''. <|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"""
prompt_for_hermes = """[
{{
  "role": "system",
  "content": "{INSTRUCTION}. Please write the appropriate SQL query using these three table schemas {all_schemas} to answer the following question. The SQL should be returned within ''' SQL query ''' " 
}},
{{
  "role": "user",
  "content": {question}
}}
]"""

# Iterate through each model
for model_key, model_value in models.items():
    responses = []

    for index, row in df.iterrows():
        qn = row['Question']
        question = json.dumps(qn)

        if model_key == "yi-34b":  # Yi model
            prompt = prompt_for_qwen.format(INSTRUCTION=INSTRUCTION, question=question)
        if model_key == "qwen-14b":  # Qwen model
            prompt = prompt_for_qwen.format(INSTRUCTION=INSTRUCTION, question=question)
        elif model_key == "openhermes2":  # Hermes model
            prompt = prompt_for_hermes.format(INSTRUCTION=INSTRUCTION, question=question)
        else:
            plain_text_question = json.loads(question)
            prompt = f"{INSTRUCTION}. Please write the appropriate SQL query using these three table schemas {all_schemas} to answer the following question. The SQL should be returned within ''' SQL query '''. {plain_text_question}"

        try:
            print(prompt)
            output = replicate.run(
                model_value,
                input={
                  "debug": False,
                #   "top_k": 50,
                  "top_p": 0.9,
                  "prompt": prompt,
                  "temperature": 0.7,
                  "max_new_tokens": 500,
                  "min_new_tokens": -1
                }
            )
            response = ""
            response_parts = []  # Initialize an empty list to collect string representations

            for item in output:
                item_str = str(item)  # Convert item to string
                response += item_str if len(item_str) == 1 else f" {item_str}"
                
            response = response.strip()
            extracted_sql = extract_sql(response) # Get only the SQL query
            valid = is_sql_valid(response) # Check if the SQL query is valid
            response = {
                "response": extracted_sql,
                "is_valid": valid
            }

        except Exception as e:
            response = f"Error: {e}"

        new_row = pd.DataFrame({'Model': [model_key], 'Question': [qn], 'Response': [response]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)

        if index % 20 == 0:  # Save every 10 questions, adjust as needed
            results_df.to_excel(f'files/{F_NAME}_results_grouped_by_model_db.xlsx', index=False, sheet_name='Sheet1')
            
results_df.to_excel(f'files/{F_NAME}_results_grouped_by_model_db.xlsx', index=False, sheet_name='Sheet1')

In [ ]:
# GPT-4 writes a SQL query

import pandas as pd
import json
import openai
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv
load_dotenv()
import os
folder_path = 'files'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

with open('config.json', 'r') as config_file:
    config = json.load(config_file)

INSTRUCTION = config['instructions']
F_NAME = config["name"]
GPT_MODEL = config["GPT_MODEL"]
INPUT_CSV_PATH = 'files/questions_db.xlsx'
OUTPUT_CSV_PATH = f'files/{F_NAME}_results_gpt4_db.xlsx'

client = OpenAI()
def show_json(obj):
    print(json.loads(obj.model_dump_json()))

assistant = client.beta.assistants.create(
    name=f"{F_NAME} AI Evaluator via reading DB",
    instructions=INSTRUCTION,
    model=GPT_MODEL,
)
show_json(assistant)

# Utility functions
def read_csv(file_path):
    return pd.read_excel(file_path)

def process_data_for_gpt(data):
    prompts = []
    for _, row in data.iterrows():
        question = row['Question']
        prompt = f"Please write the appropriate SQL query using these three table schemas {all_schemas} to answer the following question. The SQL should be returned within ''' SQL query '''.:\n\n{question}"
        prompts.append(prompt)
    return prompts

def submit_message_and_create_run(assistant_id, prompt):
    thread = client.beta.threads.create() # If you replace this globally it appends all answers to the one before.
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    return client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id), thread

def wait_on_run_and_get_response(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
    return [m.content[0].text.value for m in messages if m.role == 'assistant']

def create_output_csv(data, responses, model_name, interim_csv_path):
    new_rows = []
    for question, response in zip(data['Question'], responses):
        new_rows.append({'Model': model_name, 'Question': question, 'Response': response})
    new_data = pd.DataFrame(new_rows)
    new_data.to_excel(interim_csv_path, index=False)

data = read_csv(INPUT_CSV_PATH)
prompts = process_data_for_gpt(data)
ASSISTANT_ID = assistant.id

responses = []
for prompt in prompts:
    run, thread = submit_message_and_create_run(ASSISTANT_ID, prompt)
    response = wait_on_run_and_get_response(run, thread)
    if isinstance(response, list):
        response = ' '.join(map(str, response))
    response = response.replace("\\\\n", "\\n")
    response = response.strip()
    extracted_sql = extract_sql(response) # Get only the SQL query
    valid = is_sql_valid(response) # Check if the SQL query is valid
    response = {
        "response": extracted_sql,
        "is_valid": valid
    }
    print(response)
    responses.append(response)

create_output_csv(data, responses, GPT_MODEL, OUTPUT_CSV_PATH)